In [1]:
import pandas as pd

In [2]:
ratings_df = pd.read_csv("./data/ratings.csv")
sim_df = pd.read_csv("./data/sim.csv")
processed_df = pd.read_csv("./data/course_processed.csv")
processed_df["TITLE"] = processed_df["TITLE"].str.title()
bow_df = pd.read_csv("./data/courses_bows.csv")

In [5]:
bow_df.head()

,doc_index,doc_id,token,bow
0,0,ML0201EN,ai,2
1,0,ML0201EN,apps,2
2,0,ML0201EN,build,2
3,0,ML0201EN,cloud,1
4,0,ML0201EN,coming,1


In [11]:
grouped_df = bow_df.groupby(["doc_index", "doc_id"]).max().reset_index(drop=False)
idx_id_dict = grouped_df[["doc_id"]].to_dict()["doc_id"]
id_idx_dict = {v:k for k,v in idx_id_dict.items()}

In [18]:
def generate_recommendations_for_one_user(enrolled_course_ids,id_idx_dict, sim_matrix):
    # Create a dictionary to store your recommendation results
    res = {}
    threshold = 0.6 
    all_courses = set(idx_id_dict.values())
    unselected_course_ids = all_courses.difference(enrolled_course_ids)
    for enrolled_course in enrolled_course_ids:
        for unselect_course in unselected_course_ids:
            if enrolled_course in id_idx_dict and unselect_course in id_idx_dict:
                
                sim = 0
                idx1 = id_idx_dict[enrolled_course]
                idx2 = id_idx_dict[unselect_course]
                sim = sim_matrix[idx1][idx2]
                
                if sim > threshold:
                    if unselect_course not in res:
                        res[unselect_course] = sim
                    else:
                        if sim >= res[unselect_course]:
                            res[unselect_course] = sim
                            
    res = {k: v for k, v in sorted(res.items(), key=lambda item: item[1], reverse=True)}
    return res

In [19]:
enrolled_course_ids = ratings_df[ratings_df['user'] == 1889878]['item'].to_list()
sim_matrix = sim_df.to_numpy()

In [20]:
generate_recommendations_for_one_user(enrolled_course_ids, id_idx_dict, sim_matrix) 

{'ML0120EN': 1.0,
 'ML0120ENv3': 1.0,
 'ML0122ENv1': 0.9828731898973628,
 'CB0101EN': 0.9233805168766388,
 'excourse24': 0.7526312050490548,
 'excourse36': 0.7397041774816828,
 'excourse23': 0.7397041774816828,
 'DS0110EN': 0.7329409123199365,
 'CL0101EN': 0.7327907262791404,
 'excourse32': 0.7220184676669504,
 'excourse67': 0.7082138557765277,
 'ML0122ENv3': 0.7071067811865475,
 'excourse63': 0.6945631057877047,
 'excourse46': 0.6892532539898828,
 'excourse38': 0.6816378987999318,
 'excourse47': 0.6800651961945685,
 'excourse33': 0.6645087922452757,
 'excourse72': 0.6525351630658943,
 'excourse74': 0.6500714712738288,
 'excourse62': 0.6475015976638527,
 'excourse22': 0.6475015976638527,
 'excourse65': 0.638641451227374,
 'excourse28': 0.6237538183537886,
 'BD0145EN': 0.623543584505046,
 'excourse68': 0.616759399492821,
 'excourse60': 0.6155678409534919,
 'excourse09': 0.6083303292403595,
 'excourse25': 0.6005354126741058}